In [1]:
!pip install agentpy
!pip install seaborn


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# Model design
import agentpy as ap

# Visualization
import os
import random
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import pandas as pd
import math
import json
import numpy as np
from matplotlib import pyplot as plt



In [3]:
class ForestModel(ap.Model):

    def setup(self):
        self.time = 0
        # Create agents (trees)
        n_trees = int(self.p['Tree density'] * (self.p.size**2))
        trees = self.agents = ap.AgentList(self, n_trees)

        # Create grid (forest)
        self.forest = ap.Grid(self, [self.p.size]*2, track_empty=True)
        self.forest.add_agents(trees, random=True, empty=True)

        # Initiate a dynamic variable for all trees
        # Condition 0: Alive, 1: Burning, 2: Burned
        self.agents.condition = 0

        # Start a fire from the left side of the grid

        # Start a fire from random place on the grid
        x_axis = random.randint(0,self.p.size-2)
        y_axis = random.randint(0,self.p.size-2)

        unfortunate_trees = self.forest.agents[x_axis:x_axis+2, y_axis:y_axis+2]
        unfortunate_trees.condition = 1
        self.df = pd.DataFrame({"position":[],
                "condition":[],
                "step":[]})


    def step(self):
        self.time = self.time + 1
        # Select burning trees
        burning_trees = self.agents.select(self.agents.condition == 1)
        # Spread fire
        for tree in burning_trees:
            for neighbor in self.forest.neighbors(tree):
                if neighbor.condition == 0:
                  # Fire Parameter
                    if random.random() > self.p.fuel_parameter:
                      neighbor.condition = 1 # Neighbor starts burning
                    else:
                      neighbor.condition = 0
            tree.condition = 2 # Tree burns out


        for pos in self.forest.all:
          if str(self.forest.agents[pos].condition) == '[1]':
            row = pd.DataFrame({
                "position": [pos],
                "condition": 1,
                "step": self.time,
            })
            self.df = pd.concat([self.df, row], ignore_index=True)

        # Stop simulation if no fire is left
        if len(burning_trees) == 0:
            self.stop()

    def end(self):
        # Document a measure at the end of the simulation
        burned_trees = len(self.agents.select(self.agents.condition == 2))
        self.report('Percentage of burned trees',
                    burned_trees / len(self.agents))

        # Optionally, you can return the DataFrame
        return self.df

In [4]:
# Define parameters

parameters = {
    'Tree density': 0.6, # Percentage of grid covered by trees
    'size': 100, # Height and length of the grid
    'steps': 100,
    'fuel_parameter': 0.4
}

In [7]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.forest.attr_grid('condition')
    color_dict = {0:'#7FC97F', 1:'#d62c2c', 2:'#e5e5e5', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a forest fire\n"
                 f"Time-step: {model.t}, Trees left: "
                 f"{len(model.agents.select(model.agents.condition == 0))}")

for element in range(1,20):
    fig, ax = plt.subplots()
    model = ForestModel(parameters)
    animation = ap.animate(model, fig, ax, animation_plot)
    IPython.display.HTML(animation.to_jshtml(fps=15))

    df = model.df
    df.to_csv(f"config/fire_data/fire_data_{element}.csv")


In [13]:
def calcular_distancia(x1, y1, x2, y2):
    distancia = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distancia

def calcular_temperatura(distancia, temperatura_inicial,constante=500):
        if distancia <= 0:
            temperature = temperatura_inicial + (np.random.normal(0,1,1))
        else:
            temperature = temperatura_inicial + constante/(distancia*distancia) + (np.random.normal(0,1,1))
        return temperature[0]


def calcular_humidity(distancia, umidade_inicial, constante=5):
        if distancia <= 0:
            umidade =  umidade_inicial + (np.random.normal(0,1,1)/100)
        else:
            umidade =  umidade_inicial/np.exp(1/(constante*distancia)) + (np.random.normal(0,1,1)/100)
        return umidade[0]

In [22]:
NUM_SENSORS = '25'
sensors_data = []
for filename in os.listdir(f"config/qty_sensor_{NUM_SENSORS}/"):
    if filename.endswith(".json"):
      file = open(f"config/qty_sensor_{NUM_SENSORS}/{filename}")
      json_data = json.load(file)["sensors"]
      sensors_data.extend(json_data)
      
print(sensors_data)

[{'id': 1, 'initial_temperature': 25, 'initial_humidity': 70, 'latitude': -22.956, 'longitude': -43.224, 'pixel_position_x': 10, 'pixel_position_y': 10, 'delay_time': 10}, {'id': 2, 'initial_temperature': 25, 'initial_humidity': 70, 'latitude': -22.957, 'longitude': -43.262, 'pixel_position_x': 10, 'pixel_position_y': 30, 'delay_time': 10}, {'id': 3, 'initial_temperature': 25, 'initial_humidity': 70, 'latitude': -22.939777, 'longitude': -43.29419823, 'pixel_position_x': 10, 'pixel_position_y': 50, 'delay_time': 10}, {'id': 4, 'initial_temperature': 25, 'initial_humidity': 70, 'latitude': -22.857, 'longitude': -43.292, 'pixel_position_x': 10, 'pixel_position_y': 70, 'delay_time': 10}, {'id': 5, 'initial_temperature': 25, 'initial_humidity': 70, 'latitude': -22.956, 'longitude': -43.224, 'pixel_position_x': 10, 'pixel_position_y': 90, 'delay_time': 10}, {'id': 6, 'initial_temperature': 25, 'initial_humidity': 70, 'latitude': -22.957, 'longitude': -43.262, 'pixel_position_x': 30, 'pixel_p

In [23]:
for filename in os.listdir(f"config/fire_data"):
  sensor_ends_name = filename.split("_")[-1]
  print(filename)
  df = pd.read_csv(f"config/fire_data/{filename}")
  results = []
  for element in sensors_data:
    sensor_id = element["id"]
    pos_x = element["pixel_position_x"]
    pos_y = element["pixel_position_y"]
    temp_inicial = element["initial_temperature"]
    umid_inicial = element["initial_humidity"]
    for step in range(1,101):
      actual_burning_trees = df[df["step"]==step]
      for tree in actual_burning_trees.iterrows():
        fire_pos_x = eval(tree[1]["position"])[0]
        fire_pos_y = eval(tree[1]["position"])[1]
        dist = calcular_distancia(int(fire_pos_x),int(fire_pos_y),pos_x,pos_y)
        temp = calcular_temperatura(dist,temp_inicial)
        umid = calcular_humidity(dist,umid_inicial)
        data = {
            "step":step,
            "id":sensor_id,
            "temperature":temp,
            "humidity":umid
        }
        results.append(data)
  results_df = pd.DataFrame(results)
  results_df = results_df.groupby(["step", "id"]).agg({'temperature': 'max', 'humidity': 'min'}).reset_index()
  results_df.to_csv(f"config/qty_sensor_{NUM_SENSORS}/data/sensor_data_{sensor_ends_name}")    

fire_data_13.csv
fire_data_15.csv
fire_data_18.csv
fire_data_6.csv
fire_data_1.csv
fire_data_4.csv
fire_data_12.csv
fire_data_17.csv
fire_data_5.csv
fire_data_8.csv
fire_data_11.csv
fire_data_14.csv
fire_data_16.csv
fire_data_10.csv
fire_data_2.csv
fire_data_3.csv
fire_data_19.csv
fire_data_7.csv
fire_data_9.csv


In [7]:
df = pd.read_csv("config/fire_data/fire_data_1.csv")
for element in df.iterrows():
    fire_pos_x = eval(element[1]["position"])

In [9]:
fire_pos_x[0]

12